In [0]:
!pip install -U -q PyDrive

In [0]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!mkdir data

In [0]:
file_id = '1wWke5RAFoPXxoFYYzoWQBgqx_K8M_5RZ'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/fines.csv')

In [0]:
df = pd.read_csv('data/fines.csv')

In [7]:
df.head()

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.0,Ford,Focus,1983
1,E432XX77RUS,1,6500.0,Toyota,Camry,1996
2,7184TT36RUS,1,2100.0,Ford,Focus,1983
3,X582HE161RUS,2,2000.0,Ford,Focus,1997
4,92918M178RUS,1,5700.0,Ford,Focus,2007


In [10]:
%%time
new_col = []

for i in range(0, len(df)):
    new_col.append(
        df.iloc[i]['Fines'] / (df.iloc[i]['Refund'] * df.iloc[i]['Year'])
        )

df['Fines / Refund * Year'] = new_col

CPU times: user 708 ms, sys: 883 µs, total: 709 ms
Wall time: 709 ms


In [0]:
del df['Fines / Refund * Year']

In [12]:
%%time
new_col = []

for index, row in df.iterrows():
    new_col.append(row['Fines'] / (row['Refund'] * row['Year']))

df['Fines / Refund * Year'] = new_col

CPU times: user 200 ms, sys: 0 ns, total: 200 ms
Wall time: 201 ms


In [0]:
del df['Fines / Refund * Year']

In [14]:
%%time
df['Fines / Refund * Year'] = df.apply(
    lambda row: row['Fines'] / (row['Refund'] * row['Year']), axis=1
    )

CPU times: user 52.9 ms, sys: 935 µs, total: 53.8 ms
Wall time: 58.3 ms


In [0]:
del df['Fines / Refund * Year']

In [16]:
%%time
df['Fines / Refund * Year'] = df['Fines'] / (df['Refund'] * df['Year'])

CPU times: user 8.64 ms, sys: 854 µs, total: 9.49 ms
Wall time: 28.3 ms


In [0]:
del df['Fines / Refund * Year']

In [18]:
%%time
df['Fines / Refund * Year'] = df['Fines'].values / (
    df['Refund'].values * df['Year'].values
    )

CPU times: user 2.34 ms, sys: 0 ns, total: 2.34 ms
Wall time: 2.06 ms


In [0]:
del df['Fines / Refund * Year']

In [22]:
%%time
df.loc[df['CarNumber'] == '92918M178RUS']

CPU times: user 1.75 ms, sys: 0 ns, total: 1.75 ms
Wall time: 1.76 ms


,CarNumber,Refund,Fines,Make,Model,Year
4,92918M178RUS,1,5700.0,Ford,Focus,2007
538,92918M178RUS,2,7600.0,Ford,Focus,1982
553,92918M178RUS,1,1900.0,Ford,Focus,1982


In [0]:
df = df.set_index('CarNumber')

In [24]:
df.head()

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
Y163O8161RUS,2,3200.0,Ford,Focus,1983
E432XX77RUS,1,6500.0,Toyota,Camry,1996
7184TT36RUS,1,2100.0,Ford,Focus,1983
X582HE161RUS,2,2000.0,Ford,Focus,1997
92918M178RUS,1,5700.0,Ford,Focus,2007


In [28]:
%%time
df.loc['92918M178RUS']

CPU times: user 1.79 ms, sys: 50 µs, total: 1.84 ms
Wall time: 6.74 ms


,Refund,Fines,Make,Model,Year
CarNumber,,,,,
92918M178RUS,1,5700.0,Ford,Focus,2007
92918M178RUS,2,7600.0,Ford,Focus,1982
92918M178RUS,1,1900.0,Ford,Focus,1982


In [29]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1030 entries, Y163O8161RUS to 1478D3960RUS
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Refund  1030 non-null   int64  
 1   Fines   1030 non-null   float64
 2   Make    1030 non-null   object 
 3   Model   1010 non-null   object 
 4   Year    1030 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 257.3 KB


In [0]:
optimized = df.copy()

In [0]:
optimized['Fines'] = pd.to_numeric(optimized['Fines'], downcast='float')
optimized['Refund'] = pd.to_numeric(optimized['Refund'], downcast='integer')
optimized['Year'] = pd.to_numeric(optimized['Year'], downcast='integer')

In [32]:
optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1030 entries, Y163O8161RUS to 1478D3960RUS
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Refund  1030 non-null   int8   
 1   Fines   1030 non-null   float32
 2   Make    1030 non-null   object 
 3   Model   1010 non-null   object 
 4   Year    1030 non-null   int16  
dtypes: float32(1), int16(1), int8(1), object(2)
memory usage: 200.2 KB


In [0]:
optimized['Make'] = optimized['Make'].astype('category')
optimized['Model'] = optimized['Model'].astype('category')

In [34]:
optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1030 entries, Y163O8161RUS to 1478D3960RUS
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Refund  1030 non-null   int8    
 1   Fines   1030 non-null   float32 
 2   Make    1030 non-null   category
 3   Model   1010 non-null   category
 4   Year    1030 non-null   int16   
dtypes: category(2), float32(1), int16(1), int8(1)
memory usage: 79.9 KB


In [0]:
%reset_selective -f df

In [36]:
df

NameError: ignored